# Monte Carlo Simulation (Backend Flow)

### Import the file & extract the ticker, shares, and purchase price

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("data.csv")

print(df)

  Ticker  Shares  Purchase_Price
0   AAPL      50             180
1   GOOG      40             150


##### Additionally, calculate the portfolio value

In [23]:
tickers = df['Ticker'].to_numpy()
shares = df['Shares'].to_numpy()
purchase_price = df['Purchase_Price'].to_numpy()

portfolio_value = shares * purchase_price

print(tickers)
print(shares)
print(purchase_price)
print(portfolio_value.sum())

['AAPL' 'GOOG']
[50 40]
[180 150]
15000


### Now, we must return the historical data for the closings for our selected tickers

In [69]:
from typing import NamedTuple
from massive import RESTClient
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("MASSIVE_API_KEY")

# Load the Massive client with the proper API key

client = RESTClient(api_key=api_key)

# Get our API responses and store them

ticker_closes: dict[str, list[float]] = {}

for ticker in tickers:
    ticker_closes[ticker] = []

    for a in client.list_aggs(
        ticker,
        1,
        "day",
        "2023-01-05",
        "2025-01-08",
        sort="asc",
    ):
        # Access close attribute directly
        ticker_closes[ticker].append(a.close)

# Use it
print(f"Ticker closes: {ticker_closes['AAPL'][:]}")  # First 5 closes


MaxRetryError: HTTPSConnectionPool(host='api.massive.com', port=443): Max retries exceeded with url: /v2/aggs/ticker/AAPL/range/1/day/2023-01-05/2025-01-08?sort=asc (Caused by ResponseError('too many 429 error responses'))